In [ ]:
!pip install ultralytics
!pip install git+https://github.com/ultralytics/ultralytics.git@main

In [ ]:
import yaml
import os
import shutil
from random import seed, shuffle
import wandb
from ultralytics import YOLO

**Заметка**: Данные в датасете заранее аугментированы (случайно искажены).

Создание файла конфигурации данных в формате yaml.

In [ ]:
# Устанавливаем значение seed для генератора случайных чисел для воспроизводимости результатов
seed(1)

# Константы и конфигурации
DATASET_PATH = '/kaggle/input/welding-seams-dataset'
OUTPUT_PATH = '/kaggle/working'
CLASS_NAMES = ['adj', 'int', 'geo', 'pro', 'non']
CONFIG_FILE_PATH = os.path.join(OUTPUT_PATH, 'dataset.yaml')
DATA_SPLITS = {'train': 0.7, 'val': 0.15, 'test': 0.15}

# Функция для создания файла конфигурации данных
def create_config_file(config_path, class_names):
    data_config = {
        'train': os.path.join(OUTPUT_PATH, 'images/train'),
        'val': os.path.join(OUTPUT_PATH, 'images/val'),
        'test': os.path.join(OUTPUT_PATH, 'images/test'),
        'nc': len(class_names),
        'names': class_names
    }
    with open(config_path, 'w') as f:
        yaml.dump(data_config, f)

Создание необходимой структуры директорий, необходимой для обучения YOLOv8.

In [ ]:
# Функция для создания необходимой структуры папок
def create_directories(base_path, subdirs):
    for subdir in subdirs:
        path = os.path.join(base_path, subdir)
        if not os.path.exists(path):
            os.makedirs(path)

Разбиваем датасет на тренировочную, тестовую и проверочную выборки. Копируем файлы в обозначенные директории.

In [ ]:
# Функция для разбиения датасета на тренировочную, проверочную и тестовую выборки
def split_dataset(dataset_path):
    images = [os.path.join(dataset_path, file) for file in os.listdir(dataset_path) if file.endswith('.jpg')]
    shuffle(images)
    
    labels = [os.path.splitext(image_path)[0] + '.txt' for image_path in images]
    
    train_size = int(DATA_SPLITS['train'] * len(images))
    val_size = int(DATA_SPLITS['val'] * len(images))
    
    return {
        'train': (images[:train_size], labels[:train_size]),
        'val': (images[train_size:train_size + val_size], labels[train_size:train_size + val_size]),
        'test': (images[train_size + val_size:], labels[train_size + val_size:])
    }

# Функция для копирования файлов в целевые директории
def copy_files(files, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    for file in files:
        shutil.copy(file, target_dir)

### Основной поток выполнения программы.

In [ ]:
if __name__ == "__main__":
    # Создаем файл конфигурации данных
    create_config_file(CONFIG_FILE_PATH, CLASS_NAMES)
    
    # Создаем необходимую структуру папок для обучения YOLOv8
    create_directories(OUTPUT_PATH, [
        'images/train', 'images/val', 'images/test',
        'labels/train', 'labels/val', 'labels/test'
    ])
    
    # Разбиваем датасет на выборки
    splits = split_dataset(DATASET_PATH)
    
    # Копируем изображения и метки в соответствующие папки
    for split in splits:
        copy_files(splits[split][0], os.path.join(OUTPUT_PATH, f'images/{split}'))
        copy_files(splits[split][1], os.path.join(OUTPUT_PATH, f'labels/{split}'))
    
    print(f"Датасет успешно разбит и скопирован в соответствующие директории. Файл конфигурации создан по пути {CONFIG_FILE_PATH}")


### Обучение

Логин в `wandb`

In [ ]:
wandb.login(key='6a6d221f38aac6d82e19bb1cf2c801309be13006')

Загружаем и обучаем модель **YOLO**

In [2]:
model = YOLO("/kaggle/input/yolo8_500/other/500/2/best.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data="/kaggle/working/dataset.yaml", epochs=500)

### Оценка производительности модели

In [ ]:
# Evaluate the model's performance on the validation set
results = model.val()

# Perform object detection on an image using the model
results = model("/kaggle/input/welding-seams-dataset/1 (3).jpg")

### Экспортируем модель в формате **ONNX**

In [ ]:
# Export the model to ONNX format
success = model.export(format="onnx")

Дополнительно выполняем детекцию и выводим результаты.

In [ ]:
results = model("/kaggle/input/welding-seams-dataset/2 (13).jpg")
print(len(results))

for result in results:
    print(result.orig_img)

### Сохранение обученной модели

In [ ]:
model.export(format="onnx")